In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from six.moves.urllib.request import urlopen

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"


In [2]:
if not os.path.exists(IRIS_TRAINING):
  raw = urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'wb') as f:
    f.write(raw)

if not os.path.exists(IRIS_TEST):
  raw = urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'wb') as f:
    f.write(raw)

In [3]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [4]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_service': None, '_task_type': 'worker', '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbe337e4e10>, '_task_id': 0, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_model_dir': '/tmp/iris_model', '_save_summary_steps': 100, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000}


In [5]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True)

In [9]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 4.75227, step = 2001
INFO:tensorflow:global_step/sec: 660.108
INFO:tensorflow:loss = 7.1123, step = 2101 (0.155 sec)
INFO:tensorflow:global_step/sec: 686.144
INFO:tensorflow:loss = 5.7895, step = 2201 (0.146 sec)
INFO:tensorflow:global_step/sec: 675.67
INFO:tensorflow:loss = 5.64837, step = 2301 (0.146 sec)
INFO:tensorflow:global_step/sec: 854.758
INFO:tensorflow:loss = 6.56169, step = 2401 (0.117 sec)
INFO:tensorflow:global_step/sec: 773.286
INFO:tensorflow:loss = 6.46712, step = 2501 (0.131 sec)
INFO:tensorflow:global_step/sec: 722.848
INFO:tensorflow:loss = 3.65621, step = 2601 (0.138 sec)
INFO:tensorflow:global_step/sec: 673.045
INFO:tensorflow:loss = 4.57327, step = 2701 (0.149 sec)
INFO:tensorflow:global_step/sec: 819.136
INFO:tensorflow:loss = 7.70042, step = 2801 

In [10]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

In [11]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


INFO:tensorflow:Starting evaluation at 2018-01-06-05:32:47
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
INFO:tensorflow:Finished evaluation at 2018-01-06-05:32:48
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.966667, average_loss = 0.0777572, global_step = 4000, loss = 2.33272

Test Accuracy: 0.966667



In [12]:
# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))


INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]

